In [ ]:
from neuprint import Client
from neuprint import fetch_neurons
from neuprint import NeuronCriteria as NC
from neuprint import SynapseCriteria as SC


with open('auth_token.txt', 'r') as f:
    token = f.read()

c = Client('neuprint.janelia.org', dataset='manc:v1.0', token=token)
c.fetch_version()

In [ ]:
import pandas as pd

xl = pd.read_excel('Putative IDs.xlsx')
xl.columns = xl.loc[0,:]
xl.drop(0, inplace=True)
# drop first column
xl.drop(xl.columns[0], axis=1, inplace=True)

In [ ]:
putative_crz =  xl['Crz'].loc[~xl['Crz'].isnull()].values.astype(int)

pcrz = fetch_neurons(
    NC(
        bodyId=putative_crz,
    ),
    client=c
)[0]

In [ ]:
import bokeh.palettes
from bokeh.models import Range1d
from bokeh.plotting import figure, show, output_notebook
import numpy as np
from neuprint import fetch_synapses
from neuprint import SynapseCriteria as SC
output_notebook()

CINNABAR = '#db544b'
LAPIS = '#2d66a5'
BYZANTINE = '#D342BE'
SHAMROCK = '#33A358'


In [ ]:
import numpy as np
pcrz['ntEntropy'] = (-pcrz[['ntGlutamateProb','ntGabaProb','ntAcetylcholineProb']].values * np.log2(pcrz[['ntGlutamateProb','ntGabaProb','ntAcetylcholineProb']].values)).sum(axis=1)

In [ ]:
#pcrz.sort_values(by='ntEntropy', ascending=False, inplace=True)
pcrz[['bodyId','somaNeuromere']]

PROBABLY_CRZ = [11828, 14161, 13190, 16132, 14938, 17702, 21104, 17776] # 16945, 16407

pcrz.sort_values(by='serial')[['bodyId','serial','pre','post','predictedNtProb','somaNeuromere']]

# sns.heatmap(
#     data = [int(nm[0].split('A')[-1]) for nm in pcrz[['somaNeuromere']].values],
#     #index = pcrz[['bodyId']].values
# )

In [ ]:
from neuprint import fetch_adjacencies

_, pcrz_conn = fetch_adjacencies(
    sources = NC(
        bodyId=pcrz['bodyId'].values,
    ),
    targets = NC(
        bodyId=pcrz['bodyId'].values,
    ),
)



In [ ]:
# convert pcrz_conn into a matrix of pre x post weights
pp = pcrz_conn.pivot_table(index='bodyId_pre', columns='bodyId_post', values='weight', aggfunc='sum', fill_value=0)
pp


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

matrix = pcrz_conn.pivot_table(index='bodyId_pre', columns='bodyId_post', values='weight', aggfunc='sum', fill_value=0)
post_mat = pcrz_conn.pivot_table(index='bodyId_post', columns='bodyId_pre', values='weight', aggfunc='sum', fill_value=0)


def cosine_similarity(matrix):
    """
    Calculate the cosine similarity for a matrix.
    """
    from sklearn.metrics.pairwise import cosine_similarity

    # Calculate cosine similarity between rows
    cosine_sim_matrix = cosine_similarity(matrix)

    # Create a DataFrame with cosine similarity matrix
    cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=matrix.index, columns=matrix.index)

    return cosine_sim_df


# Calculate cosine similarity between rows
cosine_sim_matrix = cosine_similarity(matrix)

# Create a DataFrame with cosine similarity matrix
cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=matrix.index, columns=matrix.index)
cosine_sim_post_df = pd.DataFrame(cosine_similarity(post_mat), index=post_mat.index, columns=post_mat.index)

# Order the sorted matrix with hierarchical clustering
from scipy.cluster.hierarchy import linkage, dendrogram

# Calculate the linkage matrix
linkage_matrix = linkage(cosine_sim_df, 'average')
r = dendrogram(linkage_matrix, labels=cosine_sim_df.index, orientation='left', leaf_font_size=10, no_plot=True)

# Order the rows based on similarity to a reference row (for example, the first row)
reference_row = cosine_sim_df.index[0]
sorted_rows = cosine_sim_df[reference_row].sort_values(ascending=False).index

# Reorder the original DataFrame based on the sorted rows
sorted_matrix = matrix.loc[sorted_rows]

# Reorder the columns as well
sorted_matrix = sorted_matrix[sorted_rows]

# If you want to convert the sorted matrix to a NumPy array, you can use the values attribute
sorted_matrix_array = sorted_matrix.values

consine_sim_post = cosine_similarity(post_mat)
cosine_sim_post_df = pd.DataFrame(consine_sim_post, index=post_mat.index, columns=post_mat.index)

sns.heatmap(
    cosine_sim_df.loc[r['ivl']][r['ivl']],
    cmap='viridis',
    square=True,
    xticklabels=True,
    yticklabels=True,
    cbar_kws={'label': 'Cosine similarity'},
    vmin=0,
    vmax=1,
)

# sns.heatmap(
#     sorted_matrix.apply(lambda x: x/x.sum(), axis=1),
#     cmap='viridis',
#     square=True,
#     xticklabels=True,
#     yticklabels=True,
#     cbar_kws={'label': 'Number of synapses'},
#     vmin=0,
#     vmax=0.3,
# )

plt.gcf().set_size_inches(10,10)


In [ ]:
import sklearn
print(sklearn.__version__)

In [ ]:
PROBABLY_CRZ = [11828, 14161, 13190, 16132, 14938, 17702, 21104, 17776] # 16945, 16407
#PROBABLY_CRZ = [11828, 17702, 21104, 17776, 16132, 14938, 13190, 14161]
#PROBABLY_CRZ = [14161, 16132, 14938, 17702, 13190, 11828,]
PROBABLY_CRZ = [14161, 11828, 13190, 16132 ,] #17702, 14938] 
CRZ_PLUS =[14161, 11828, 13190, 16132 , 17702, 14938] 
#PROBABLY_CRZ = [11828, 13190, 14938, 17702, ]


_, crz_to_crz = fetch_adjacencies(
    sources = NC(
        bodyId=PROBABLY_CRZ,
    ),
    # targets = NC(
    #     bodyId=PROBABLY_CRZ,
    # ),
)

crzs = fetch_neurons(
    NC(
        bodyId=PROBABLY_CRZ,
    ),
    client=c
)[0]

matrix = crz_to_crz.pivot_table(index='bodyId_pre', columns='bodyId_post', values='weight', aggfunc='sum', fill_value=0)

# sort by cosine similarity
cosine_sim_matrix = cosine_similarity(matrix)
cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=matrix.index, columns=matrix.index)
reference_row = cosine_sim_df.index[0]
sorted_rows = cosine_sim_df[reference_row].sort_values(ascending=False).index
sorted_matrix = matrix.loc[sorted_rows]
sorted_matrix = sorted_matrix[sorted_rows]

sns.heatmap(
    cosine_sim_df[sorted_rows].loc[sorted_rows],
    #matrix[PROBABLY_CRZ].loc[PROBABLY_CRZ],#.values,#sorted_matrix,
    #matrix.apply(lambda x: x/x.sum(), axis=1),
    cmap='viridis',
    square=True,
    xticklabels=True,
    yticklabels=True,
    cbar_kws={'label': 'Number of synapses'},
    vmin=0,
    #vmax=0.3,
)

# plt.savefig('crz_crz_cosine_distance.svg', format='svg')

# Let's do hierarchical clustering

from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist, squareform

# Calculate the distance matrix
distance_matrix = pdist(matrix/matrix.sum(axis=0), metric='euclidean')

# Convert the distance matrix to a square-form distance matrix
distance_sqmatrix = squareform(distance_matrix)

# Calculate the linkage matrix
Z = linkage(distance_sqmatrix, method='ward')

# Plot the dendrogram
plt.figure(figsize=(10, 10))
dendrogram(Z, labels=matrix.index, orientation='left', leaf_font_size=10)
plt.xlabel('Distance')
plt.ylabel('Neuron ID')
plt.title('Dendrogram of CRZ neurons')
#plt.show()

# plt.gcf().savefig('crz_crz_dendrogram.svg', format='svg')

In [ ]:
efferent_df, crz_to_efferent = fetch_adjacencies(
    sources = NC(
        bodyId=PROBABLY_CRZ,
        #serial = [11828],
    ),
    targets = NC(
        type = 'EN.*',
        regex=True,
    ),
)

matrix = crz_to_efferent.pivot_table(index='bodyId_pre', columns='bodyId_post', values='weight', aggfunc='sum', fill_value=0)

sns.heatmap(
    matrix,
    cmap='viridis',
    square=True,
    xticklabels=True,
    yticklabels=True,
    cbar_kws={'label': 'Number of synapses'},
    vmin=0,
    #vmax=0.3,
)

plt.gcf().set_size_inches(10,10)


In [ ]:
crz_post_ns, crz_post = fetch_adjacencies(
    sources = NC(
        bodyId=PROBABLY_CRZ,
    ),
    min_total_weight=5,
)

In [ ]:

matrix = crz_post.pivot_table(index='bodyId_pre', columns='bodyId_post', values='weight', aggfunc='sum', fill_value=0)

plt.gcf().set_size_inches(10,2)

main_targets = matrix.aggregate('sum', axis=0).sort_values(ascending=False).index
#Normalize the rows
normed_matrix = matrix.apply(lambda x: x/x.sum(), axis=1)


sns.heatmap(
    normed_matrix[main_targets].loc[PROBABLY_CRZ],
    cmap='viridis',
    square=False,
    xticklabels=False,
    yticklabels=True,
    cbar_kws={'label': 'Fraction of synapses'},
    vmin=0,
    #vmax=0.3,
)

#plt.gcf().savefig('crz_main_targets.svg', format='svg')

# plt.plot(
#     np.arange(len(main_targets)).reshape(-1,1),
#     matrix[main_targets].T.sum(axis=1).values,
#     'o',
# )


In [ ]:
summed_out = matrix[main_targets].T.sum(axis=1)
plt.plot(
    np.arange(len(main_targets)).reshape(-1,1),
    summed_out.values,
    'ok',
)

CrzR = [11092, 12614, 168470, 1200, 11765, 11469, 11584]

outputs_df = fetch_neurons(NC(bodyId = summed_out.index))[0][['bodyId', 'ntGlutamateProb', 'ntGabaProb', 'ntAcetylcholineProb', 'predictedNtProb', 'predictedNt', 'pre', 'post', 'type', 'outputRois']]

CINNABAR = '#db544b'

# # plot uncertain neurons in pink
uncertain = outputs_df[outputs_df['predictedNtProb'] <= 0.7]['bodyId'].values
uncertain_locvals= np.array([(summed_out.index.get_loc(cell), summed_out[cell]) for cell in uncertain]).T
plt.plot(
    uncertain_locvals[0],
    uncertain_locvals[1],
    'o',
    color = BYZANTINE,
)

#plot descending neurons in yellow
descending = outputs_df[
    outputs_df['type'].notna() 
    & (
        outputs_df['type'].str.contains('MN.*', regex=True)
        | outputs_df['type'].str.contains('EN.*', regex=True)
    )
    & ~(
        outputs_df['bodyId'].isin(CRZ_PLUS)
        | outputs_df['bodyId'].isin(CrzR)
    )]['bodyId'].values
descending_locvals= np.array([(summed_out.index.get_loc(cell), summed_out[cell]) for cell in descending]).T
plt.plot(
    descending_locvals[0],
    descending_locvals[1],
    'o',
    color = '#CEC019',
)

# plot putative CrzR in red

endf, _ = fetch_neurons(
    NC(
        bodyId = CrzR
    ),
    client=c
)

en00 = [cell for cell in summed_out.index.values if cell in endf['bodyId'].values]
en_locvals= np.array([(summed_out.index.get_loc(cell), summed_out[cell]) for cell in en00]).T
plt.plot(
    en_locvals[0],
    en_locvals[1],
    'o',
    color = CINNABAR,
)

# plot ascending neurons in blue
ascending = outputs_df[
    outputs_df['type'].notna() 
    & (
        outputs_df['type'].str.contains('AN.*', regex=True)
    )]['bodyId'].values
ascending_locvals= np.array([(summed_out.index.get_loc(cell), summed_out[cell]) for cell in ascending]).T
plt.plot(
    ascending_locvals[0],
    ascending_locvals[1],
    'o',
    color = LAPIS,
)

local_neurons = outputs_df[
    ~(
        outputs_df['type'].str.contains('AN.*', regex=True)
        | outputs_df['type'].str.contains('MN.*', regex=True)
        | outputs_df['type'].str.contains('EN.*', regex=True)
        | outputs_df['bodyId'].isin(CRZ_PLUS)
        | outputs_df['bodyId'].isin(CrzR)
    ) & outputs_df['outputRois'].apply(lambda x: all(item == 'ANm' or item.startswith('Ab') for item in x))]

# plot local neurons in green
gaba_local = local_neurons[
    local_neurons['ntGabaProb'] > 0.7]['bodyId'].values
gaba_local_locvals= np.array([(summed_out.index.get_loc(cell), summed_out[cell]) for cell in gaba_local]).T

choline_local = local_neurons[
    local_neurons['ntAcetylcholineProb'] > 0.7]['bodyId'].values
choline_local_locvals= np.array([(summed_out.index.get_loc(cell), summed_out[cell]) for cell in choline_local]).T

glutamate_local = local_neurons[
    local_neurons['ntGlutamateProb'] > 0.7]['bodyId'].values
glutamate_local_locvals= np.array([(summed_out.index.get_loc(cell), summed_out[cell]) for cell in glutamate_local]).T

unknown_local = local_neurons[
    local_neurons['predictedNtProb'] <= 0.7]['bodyId'].values
unknown_local_locvals= np.array([(summed_out.index.get_loc(cell), summed_out[cell]) for cell in unknown_local]).T

plt.plot(
    gaba_local_locvals[0],
    gaba_local_locvals[1],
    'o',
    color = SHAMROCK,
)

# plot putative_crz neurons in seafoam
crz_locvals= np.array([(summed_out.index.get_loc(cell), summed_out[cell]) for cell in PROBABLY_CRZ]).T
plt.plot(
    crz_locvals[0],
    crz_locvals[1],
    'o',
    color = '#00A8A3'
)


plt.gca().spines['bottom'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().set_xticks([])
plt.gca().set_xlim(-1, 100)
plt.gca().set_ylabel("Total synapses")
plt.gcf().set_size_inches(10,2)



In [ ]:
from matplotlib.patches import Rectangle

bar_plot_f, bar_plot_x = plt.subplots(1,1, figsize=(10,2))

bar_plot_x.set_ylim(0, 1)
bar_plot_x.set_xlim(0, 1)

x, y = 0, 0

width = summed_out.loc[descending].sum() / summed_out.sum()
print(f"Descending: {width}")

bar_plot_x.add_patch(
    Rectangle((x,y), width, 1, color = '#CEC019')
)
x+= width

width = summed_out.loc[ascending].sum() / summed_out.sum()
print(f"Ascending: {width}")

bar_plot_x.add_patch(
    Rectangle((x,y), width, 1, color = LAPIS)
)
x+=width

width = summed_out.loc[en00].sum() / summed_out.sum()
print(f"CrzR: {width}")

bar_plot_x.add_patch(
    Rectangle((x,y), width, 1, color = CINNABAR)
)
x+= width

width = summed_out.loc[CRZ_PLUS].sum() / summed_out.sum()
print(f"Crz+: {width}")

bar_plot_x.add_patch(
    Rectangle((x,y), width, 1, color = '#00A8A3')
)
x+= width

width = summed_out.loc[unknown_local].sum() / summed_out.sum()
print(f"Unknown: {width}")

bar_plot_x.add_patch(
    Rectangle((x,y), width, 1, color = BYZANTINE)
)
x+= width

width = summed_out.loc[gaba_local].sum() / summed_out.sum()
print(f"GABA: {width}")

bar_plot_x.add_patch(
    Rectangle((x,y), width, 1, color = SHAMROCK)
)
x+= width

width = summed_out.loc[glutamate_local].sum() / summed_out.sum()
print(f"Glutamate: {width}")

bar_plot_x.add_patch(
    Rectangle((x,y), width, 1, color = '#CE200F')
)

x+= width

width = summed_out.loc[choline_local].sum() / summed_out.sum()
print(f"Choline: {width}")

bar_plot_x.add_patch(
    Rectangle((x,y), width, 1, color = '#5E24CE')
)

x+= width

bar_plot_x.set_xticks([])
bar_plot_x.set_yticks([])
print(width)

#bar_plot_f.savefig('crz_output_bar_plot.svg', format='svg')



In [ ]:
roi_mesh = c.fetch_roi_mesh('ANm')


In [ ]:
def plot_neurons(source_df, idx : int, flattened_ax : int = 1, on_p = None, weighted_by : float = 1):
    if on_p is None:
        p = figure(
            match_aspect = True
        )
        p.y_range.flipped = False
    else:
        p = on_p
    try:
        skelee = c.fetch_skeleton(source_df.iloc[idx]['bodyId'])

        skelee.radius = skelee.radius.astype(float)/(20)
        # Join parent/child nodes for plotting as line segments below.
        # (Using each row's 'link' (parent) ID, find the row with matching rowId.)
        skelee = skelee.merge(skelee, 'inner',
                                left_on=['link'],
                                right_on=['rowId'],
                                suffixes=['_child', '_parent'])
        axes = [x for x in range(3) if x != flattened_ax]
        verts = np.array([
            [float(row.split(' ')[axes[0]+1]),float(row.split(' ')[axes[1]+1])]
            for row in str(roi_mesh).split('\\n') if row.startswith('v')
        ])

        speckle_size = 1.0
        p.scatter(verts[:,0],verts[:,1],size=speckle_size,line_alpha=0.0, fill_color = (0,0,0), fill_alpha = 0.5)

        str_labels = ['x', 'y', 'z']

        as_str = [str_labels[i] for i in axes]

        # Plot skeleton segments (in 2D)
        p.segment(
            x0=f'{as_str[0]}_child', x1=f'{as_str[0]}_parent',
            y0=f'{as_str[1]}_child', y1=f'{as_str[1]}_parent',
            line_width = 'radius_parent',
            alpha = 0.5*weighted_by,
            color = LAPIS,
            source=skelee
        )
        

        p.renderers.insert(0,p.renderers.pop())

        lims = {
            'x' : Range1d(18000,30000),
            'y' : Range1d(0,20000),
            'z' : Range1d(0,20000)
        }

        if as_str[0] in lims:
            p.x_range = lims[as_str[0]]
        if as_str[1] in lims:
            p.y_range = lims[as_str[1]]

        #p.y_range = Range1d(0,20000)
        #p.x_range = Range1d(18000,30000)
        p.xgrid.visible = False
        p.ygrid.visible = False
        p.axis.visible = False
        #return p
    except:
        pass
    finally:
        return p

from bokeh.plotting import gridplot

from bokeh.io import export_png

crzr_df = fetch_neurons(
    NC(
        bodyId = gaba_local
    ),
    client=c
)[0]

#output_file(filename='/Users/stephen/Desktop/Data/SICode/manc/some_neurons.png')
# make a grid
normed = summed_out[crzr_df['bodyId']] / summed_out[crzr_df['bodyId']].max()
on_ps = [plot_neurons(crzr_df, 0, 0, weighted_by = normed.iloc[0]), plot_neurons(crzr_df, 0, 1, weighted_by = normed.iloc[0]), plot_neurons(crzr_df, 0, 2, weighted_by =normed.iloc[0])]
grid = gridplot([[plot_neurons(crzr_df,idx, k, on_ps[k], weighted_by = normed.iloc[idx]) for k in range(3)] for idx in range(1, len(crzr_df))])
#grid = gridplot([[plot_neurons(crzr_df,idx, 0, on_p) for k in range(3)] for idx in range(len(crzr_df))])

#show(grid)

export_png(grid, filename = f'gaba_downstream_of_crz.png')


In [ ]:
summed_out[crzr_df['bodyId']] / summed_out[crzr_df['bodyId']].max()

In [ ]:
normed.sort_values(ascending=False)

In [ ]:
crz_pre_ns, crz_pre = fetch_adjacencies(
    targets = NC(
        bodyId=PROBABLY_CRZ,
    ),
    min_total_weight=5,
)

In [ ]:

matrix = crz_pre.pivot_table(index='bodyId_post', columns='bodyId_pre', values='weight', aggfunc='sum', fill_value=0)

plt.gcf().set_size_inches(10,2)

main_targets = matrix.aggregate('sum', axis=0).sort_values(ascending=False).index
#Normalize the rows
normed_matrix = matrix.apply(lambda x: x/x.sum(), axis=1)


sns.heatmap(
    normed_matrix[main_targets].loc[PROBABLY_CRZ],
    cmap='viridis',
    square=False,
    xticklabels=False,
    yticklabels=True,
    cbar_kws={'label': 'Fraction of synapses'},
    vmin=0,
    #vmax=0.3,
)

#plt.gcf().savefig('crz_main_targets.svg', format='svg')

# plt.plot(
#     np.arange(len(main_targets)).reshape(-1,1),
#     matrix[main_targets].T.sum(axis=1).values,
#     'o',
# )


In [ ]:
summed_in = matrix[main_targets].T.sum(axis=1)
plt.plot(
    np.arange(len(main_targets)).reshape(-1,1),
    summed_in.values,
    'ok',
)

inputs_df = fetch_neurons(NC(bodyId = summed_in.index))[0][['bodyId', 'ntGlutamateProb', 'ntGabaProb', 'ntAcetylcholineProb', 'predictedNtProb', 'predictedNt', 'pre', 'post', 'type', 'outputRois']]

CINNABAR = '#db544b'

# # plot uncertain neurons in pink
uncertain = inputs_df[inputs_df['predictedNtProb'] <= 0.7]['bodyId'].values
uncertain_locvals= np.array([(summed_in.index.get_loc(cell), summed_in[cell]) for cell in uncertain]).T
plt.plot(
    uncertain_locvals[0],
    uncertain_locvals[1],
    'o',
    color = BYZANTINE,
)

#plot descending neurons in yellow
# descending = inputs_df[
#     inputs_df['type'].notna() 
#     & (
#         inputs_df['type'].str.contains('MN.*', regex=True)
#         | inputs_df['type'].str.contains('EN.*', regex=True)
#     )
#     & ~(
#         inputs_df['bodyId'].isin(CRZ_PLUS)
#         | inputs_df['bodyId'].isin(CrzR)
#     )]['bodyId'].values
# descending_locvals= np.array([(summed_in.index.get_loc(cell), summed_in[cell]) for cell in descending]).T
# plt.plot(
#     descending_locvals[0],
#     descending_locvals[1],
#     'o',
#     color = '#CEC019',
# )

# plot putative CrzR in red

CrzR = [11092, 12614, 168470, 1200, 11765, 11469, 11584]

# endf, _ = fetch_neurons(
#     NC(
#         bodyId = CrzR
#     ),
#     client=c
# )

# en00 = [cell for cell in summed_in.index.values if cell in endf['bodyId'].values]
# en_locvals= np.array([(summed_in.index.get_loc(cell), summed_in[cell]) for cell in en00]).T
# plt.plot(
#     en_locvals[0],
#     en_locvals[1],
#     'o',
#     color = CINNABAR,
# )

# plot ascending neurons in blue
# ascending = inputs_df[
#     inputs_df['type'].notna() 
#     & (
#         inputs_df['type'].str.contains('AN.*', regex=True)
#     )]['bodyId'].values
# ascending_locvals= np.array([(summed_in.index.get_loc(cell), summed_in[cell]) for cell in ascending]).T
# plt.plot(
#     ascending_locvals[0],
#     ascending_locvals[1],
#     'o',
#     color = LAPIS,
# )

local_neurons = inputs_df[
    ~(
        inputs_df['type'].str.contains('AN.*', regex=True)
        | inputs_df['type'].str.contains('MN.*', regex=True)
        | inputs_df['type'].str.contains('EN.*', regex=True)
        | inputs_df['bodyId'].isin(CRZ_PLUS)
        | inputs_df['bodyId'].isin(CrzR)
        | inputs_df['bodyId'].isin(summed_in.index.values[:4])
    ) & inputs_df['outputRois'].apply(lambda x: all(item == 'ANm' or item.startswith('Ab') for item in x))]

# plot local neurons in green
gaba_local = local_neurons[
    local_neurons['ntGabaProb'] > 0.7]['bodyId'].values
gaba_local_locvals= np.array([(summed_in.index.get_loc(cell), summed_in[cell]) for cell in gaba_local]).T

choline_local = local_neurons[
    local_neurons['ntAcetylcholineProb'] > 0.7]['bodyId'].values
choline_local_locvals= np.array([(summed_in.index.get_loc(cell), summed_in[cell]) for cell in choline_local]).T

glutamate_local = local_neurons[
    local_neurons['ntGlutamateProb'] > 0.7]['bodyId'].values
glutamate_local_locvals= np.array([(summed_in.index.get_loc(cell), summed_in[cell]) for cell in glutamate_local]).T

unknown_local = local_neurons[
    local_neurons['predictedNtProb'] <= 0.7]['bodyId'].values
unknown_local_locvals= np.array([(summed_in.index.get_loc(cell), summed_in[cell]) for cell in unknown_local]).T

plt.plot(
    gaba_local_locvals[0],
    gaba_local_locvals[1],
    'o',
    color = SHAMROCK,
)

# plot putative_crz neurons in seafoam
crz_locvals= np.array([(summed_in.index.get_loc(cell), summed_in[cell]) for cell in CRZ_PLUS]).T
plt.plot(
    crz_locvals[0],
    crz_locvals[1],
    'o',
    color = '#00A8A3'
)


plt.gca().spines['bottom'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().set_xticks([])
plt.gca().set_xlim(-1, 100)
plt.gca().set_ylabel("Total synapses")
plt.gcf().set_size_inches(10,2)


In [ ]:
crz_top_in = fetch_neurons(NC(bodyId=summed_in.index.values[:12]))[0]

In [ ]:
def plot_neurons(source_df, idx : int, flattened_ax : int = 1, on_p = None):

        skelee = c.fetch_skeleton(source_df.iloc[idx]['bodyId'])

        skelee.radius = skelee.radius.astype(float)/(20)
        # Join parent/child nodes for plotting as line segments below.
        # (Using each row's 'link' (parent) ID, find the row with matching rowId.)
        skelee = skelee.merge(skelee, 'inner',
                                left_on=['link'],
                                right_on=['rowId'],
                                suffixes=['_child', '_parent'])
        if on_p is None:
            p = figure(
                match_aspect = True
            )
            p.y_range.flipped = False
        else:
            p = on_p

        axes = [x for x in range(3) if x != flattened_ax]
        verts = np.array([
            [float(row.split(' ')[axes[0]+1]),float(row.split(' ')[axes[1]+1])]
            for row in str(roi_mesh).split('\\n') if row.startswith('v')
        ])

        speckle_size = 1.0
        p.scatter(verts[:,0],verts[:,1],size=speckle_size,line_alpha=0.0, fill_color = (0,0,0), fill_alpha = 0.5)

        str_labels = ['x', 'y', 'z']

        as_str = [str_labels[i] for i in axes]

        # Plot skeleton segments (in 2D)
        p.segment(
            x0=f'{as_str[0]}_child', x1=f'{as_str[0]}_parent',
            y0=f'{as_str[1]}_child', y1=f'{as_str[1]}_parent',
            line_width = 'radius_parent',
            alpha = 0.5,
            color = SHAMROCK,
            source=skelee
        )
        

        p.renderers.insert(0,p.renderers.pop())

        lims = {
            'x' : Range1d(18000,30000),
            'y' : Range1d(0,20000),
            'z' : Range1d(0,20000)
        }

        if as_str[0] in lims:
            p.x_range = lims[as_str[0]]
        if as_str[1] in lims:
            p.y_range = lims[as_str[1]]

        #p.y_range = Range1d(0,20000)
        #p.x_range = Range1d(18000,30000)
        p.xgrid.visible = False
        p.ygrid.visible = False
        p.axis.visible = False
        return p

from bokeh.plotting import gridplot

from bokeh.io import export_png

crzr_df = fetch_neurons(
    NC(
        bodyId = summed_in[:4].index.values
    ),
    client=c
)[0]

#output_file(filename='/Users/stephen/Desktop/Data/SICode/manc/some_neurons.png')
# make a grid
on_ps = [plot_neurons(crzr_df, 0, 0), plot_neurons(crzr_df, 0, 1), plot_neurons(crzr_df, 0, 2)]
grid_time = gridplot([[plot_neurons(crzr_df,idx, k, on_ps[k]) for k in range(3)] for idx in range(1,len(crzr_df))])#len(crzr_df))])
#grid = gridplot([[plot_neurons(crzr_df,idx, 0, on_p) for k in range(3)] for idx in range(len(crzr_df))])

show(grid_time)

export_png(grid_time, filename = 'here_Dh44.png')


In [ ]:
from matplotlib.patches import Rectangle

bar_plot_f, bar_plot_x = plt.subplots(1,1, figsize=(10,2))

bar_plot_x.set_ylim(0, 1)
bar_plot_x.set_xlim(0, 1)

x, y = 0, 0

width = summed_in.iloc[:4].sum() / summed_in.sum()
print(f"Dh44-like: {width}")

bar_plot_x.add_patch(
    Rectangle((x,y), width, 1, color = SHAMROCK)
)
x+= width

# width = summed_in.loc[ascending].sum() / summed_in.sum()
# print(f"Ascending: {width}")

# bar_plot_x.add_patch(
#     Rectangle((x,y), width, 1, color = LAPIS)
# )
# x+=width

# width = summed_in.loc[en00].sum() / summed_in.sum()
# print(f"CrzR: {width}")

# bar_plot_x.add_patch(
#     Rectangle((x,y), width, 1, color = CINNABAR)
# )
# x+= width

width = summed_in.loc[CRZ_PLUS].sum() / summed_in.sum()
print(f"Crz+: {width}")

bar_plot_x.add_patch(
    Rectangle((x,y), width, 1, color = '#00A8A3')
)
x+= width

width = summed_in.loc[unknown_local].sum() / summed_in.sum()

print(f"Unknown: {width}")

bar_plot_x.add_patch(
    Rectangle((x,y), width, 1, color = BYZANTINE)
)
x+= width

width = summed_in.loc[gaba_local].sum() / summed_in.sum()
print(f"GABA: {width}")

bar_plot_x.add_patch(
    Rectangle((x,y), width, 1, color = CINNABAR)
)
x+= width

width = summed_in.loc[glutamate_local].sum() / summed_in.sum()
print(f"Glutamate: {width}")

bar_plot_x.add_patch(
    Rectangle((x,y), width, 1, color = '#CE200F')
)

x+= width

width = summed_in.loc[choline_local].sum() / summed_in.sum()
print(f"Choline: {width}")

bar_plot_x.add_patch(
    Rectangle((x,y), width, 1, color = '#5E24CE')
)

x+= width

print(1-x)

bar_plot_x.set_xticks([])
bar_plot_x.set_yticks([])

bar_plot_f.savefig('crz_input_bar_plot.svg', format='svg')



In [ ]:
summed_in.iloc[4:]

In [ ]:
print(bokeh.__version__)